#Fake data creation

In [1]:
!pip install faker

     |████████████████████████████████| 1.2 MB 7.9 MB/s 


In [2]:
from faker import Faker
import random
import pandas as pd


faker = Faker()
rating=[0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
data=[]
pid=[]
################product_info
for i in range(100):
  temp={}
  temp['prodId']=i
  temp['name']=faker.sentence(nb_words=3,variable_nb_words=True)[:-1]
  temp['description']=faker.sentence()
  temp['category']=random.randrange(8)
  data.append(temp)
prod_info=pd.DataFrame(data)
pid=prod_info['prodId'].to_list()
prod_info.to_csv('prod_info.csv')

################rating review
data=[]
for i in range(2000):
  temp={}
  temp['userId']=random.randrange(100)
  temp['prodId']=random.choice(pid)
  temp['product review']=faker.sentence()
  temp['rating']=random.choice(rating)
  # temp['rating count']=random.randrange(1000)
  data.append(temp)
rat_info=pd.DataFrame(data)
rat_info.to_csv('rat_info.csv')

#################users history
data=[]
for i in range(1000):
  temp={}
  temp['timestemp']=i
  temp['userId']=random.randrange(100)
  temp['prodId']=random.choice(pid)
  data.append(temp)
uhis_info=pd.DataFrame(data)
uhis_info.to_csv('uhis_info.csv')
# """id
# name
# category
# rating
# reviews
# description"""


In [3]:
prod_info.head(10)

,prodId,name,description,category
0,0,Manager easy service nearly,Seem until physical more professional.,6
1,1,Dinner various among pick,Include why collection build hold house on rel...,4
2,2,While large,Occur painting front goal present.,0
3,3,Six,During fall along reality name recently medical.,7
4,4,Young me international,Debate daughter moment outside message black p...,3
5,5,Start knowledge friend,Add civil baby decision.,1
6,6,Own important financial,Thought town apply moment.,3
7,7,Smile partner argue,Specific never point sport quality dark.,3
8,8,Light,Help boy pretty stand director front beat staff.,0
9,9,State we during,Save require forget inside.,1


In [4]:
rat_info.head(10)

,userId,prodId,product review,rating
0,14,6,Power although sign radio western family.,5.0
1,64,32,Half like air hope space table art.,3.0
2,26,53,Expect state sort buy next.,2.0
3,52,70,Give rise how.,2.0
4,97,89,Author though table sit main just ready smile.,0.5
5,0,47,I money parent right.,0.5
6,22,3,Job quality answer fill.,3.5
7,45,14,May fish cold.,0.5
8,66,14,Operation training culture book central remain.,5.0
9,67,30,Tough church already prevent various stuff.,4.5


In [5]:
uhis_info.head(10)

,timestemp,userId,prodId
0,0,80,74
1,1,60,75
2,2,94,7
3,3,73,52
4,4,24,1
5,5,88,17
6,6,18,1
7,7,42,18
8,8,79,8
9,9,1,43


In [6]:
prod_info['prodId'].nunique()

100

In [7]:
rat_info.groupby('prodId')['userId'].nunique()

prodId
0     17
1     22
2     18
3     18
4     15
      ..
95    18
96    16
97    19
98    21
99    10
Name: userId, Length: 100, dtype: int64

In [8]:
uhis_info.groupby('prodId').nunique()

,timestemp,userId
prodId,,
0,13,12
1,13,12
2,13,13
3,14,12
4,10,10
...,...,...
95,5,5
96,16,15
97,10,8


#Popular products and counts of urls visited by user

count visited url per product (popular product)

In [9]:
import pandas as pd
prod_info_path='prod_info.csv'
rat_info_path='rat_info.csv'
uhis_info_path='uhis_info.csv'
prod_info=pd.read_csv(prod_info_path,index_col=0)
rat_info=pd.read_csv(rat_info_path,index_col=0)
uhis_info=pd.read_csv(uhis_info_path,index_col=0)


In [10]:
uhis_info['product_visitor_count']=uhis_info.groupby('prodId')['prodId'].transform('count')
#df=pd.read_csv('sample-data.csv', low_memory=False)

In [11]:
uhis_info_p=uhis_info.drop_duplicates(subset='prodId', keep="last")
prod_info= prod_info.merge(uhis_info_p[['prodId','product_visitor_count']],on=['prodId'],how='outer')

In [12]:
prod_info['product_visitor_count']=prod_info['product_visitor_count'].fillna(0)

In [13]:
prod_info['product_visitor_count'] = (prod_info['product_visitor_count'] - prod_info['product_visitor_count'].min()) / (prod_info['product_visitor_count'].max() - prod_info['product_visitor_count'].min())    


In [14]:
uhis_info=uhis_info.drop(['product_visitor_count'],axis=1)

prod_info have new column with visiter count  
while uhistInfo have update column with prodId with normalization 

#Weighted Avarage

In [15]:
rat_prod=rat_info.groupby(["prodId"],as_index=False).agg(rat_count=pd.NamedAgg(column="rating", aggfunc="count"),rat_avrg=pd.NamedAgg(column="rating", aggfunc="mean"))
# rat_info_data=rat_info.groupby(["pid"]).agg(rat_count=pd.NamedAgg(column="rating", aggfunc="count"),rat_avrg=pd.NamedAgg(column="rating", aggfunc="mean"))
# rat_info['rat_avrg']=rat_info.groupby(['pid']).mean()

Formula:
(v/(v+m) * R) + (m/(m+v) * C)

---


v is the number of ratings  
m is the minimum rating count required to be listed in the data  
R is the average rating of the product  
C is the mean vote across the whole data.

In [16]:
C = rat_prod['rat_avrg'].mean()
C

2.837896526676891

In [17]:
m = rat_prod['rat_count'].quantile(0.90)
m

26.0

only qulified products

In [18]:
def weighted_rating(x, m=m, C=C):
    v = x['rat_count']
    R = x['rat_avrg']
    return (v/(v+m) * R) + (m/(m+v) * C)
rat_prod['rscore'] = rat_prod.apply(weighted_rating, axis=1)

In [19]:
prod_info_final=prod_info.merge(rat_prod,on='prodId')

#Sentiment of text review

huggingface

In [20]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.7 MB/s 
     |████████████████████████████████| 636 kB 53.0 MB/s 
     |████████████████████████████████| 895 kB 53.6 MB/s 
     |████████████████████████████████| 3.3 MB 50.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import re

# Preprocess text (username and link placeholders)
def preprocess(text):
    res_text = re.sub(r'[^\w\s]', '', text)
    return res_text

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
print(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)


cardiffnlp/twitter-roberta-base-sentiment


In [22]:

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


In [23]:
def sentiment_apply(row):
  text = row['product review']
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  row['negative']=np.round(float(scores[0]), 4)
  row['neutral']=np.round(float(scores[1]), 4)
  row['positive']=np.round(float(scores[2]), 4)
  return row




# rat_info
rat_info=rat_info.apply(sentiment_apply,axis=1)

In [24]:
rat_info_temp=rat_info.groupby('prodId')['negative','neutral','positive'].mean() 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [25]:
rat_info_temp

,negative,neutral,positive
prodId,,,
0,0.138630,0.724430,0.136950
1,0.149925,0.749217,0.100883
2,0.098733,0.801756,0.099511
3,0.170311,0.719532,0.110142
4,0.144188,0.757131,0.098675
...,...,...,...
95,0.085879,0.775926,0.138184
96,0.109412,0.782488,0.108135
97,0.077640,0.705680,0.216675


In [26]:
prod_info_final=prod_info_final.merge(rat_info_temp,on='prodId')

In [27]:
prod_info_final

,prodId,name,description,category,product_visitor_count,rat_count,rat_avrg,rscore,negative,neutral,positive
0,0,Manager easy service nearly,Seem until physical more professional.,6,0.666667,20,2.350000,2.625768,0.138630,0.724430,0.136950
1,1,Dinner various among pick,Include why collection build hold house on rel...,4,0.666667,24,3.125000,2.975706,0.149925,0.749217,0.100883
2,2,While large,Occur painting front goal present.,0,0.666667,18,2.916667,2.870121,0.098733,0.801756,0.099511
3,3,Six,During fall along reality name recently medical.,7,0.733333,19,2.947368,2.884118,0.170311,0.719532,0.110142
4,4,Young me international,Debate daughter moment outside message black p...,3,0.466667,16,3.281250,3.006793,0.144188,0.757131,0.098675
...,...,...,...,...,...,...,...,...,...,...,...
95,95,Care career,Word month trial.,3,0.133333,19,2.789474,2.817451,0.085879,0.775926,0.138184
96,96,Everyone not forward,Middle prevent discover.,6,0.866667,17,2.411765,2.669426,0.109412,0.782488,0.108135
97,97,Choice need,Themselves like again way.,6,0.466667,20,2.700000,2.777942,0.077640,0.705680,0.216675
98,98,Need sit father,Left clear exist piece because full.,6,0.466667,23,2.543478,2.699700,0.147761,0.679687,0.172539




# Content based filtering

In [28]:
prod_info_final.reset_index(inplace = True, drop = True)
prod_info_final

,prodId,name,description,category,product_visitor_count,rat_count,rat_avrg,rscore,negative,neutral,positive
0,0,Manager easy service nearly,Seem until physical more professional.,6,0.666667,20,2.350000,2.625768,0.138630,0.724430,0.136950
1,1,Dinner various among pick,Include why collection build hold house on rel...,4,0.666667,24,3.125000,2.975706,0.149925,0.749217,0.100883
2,2,While large,Occur painting front goal present.,0,0.666667,18,2.916667,2.870121,0.098733,0.801756,0.099511
3,3,Six,During fall along reality name recently medical.,7,0.733333,19,2.947368,2.884118,0.170311,0.719532,0.110142
4,4,Young me international,Debate daughter moment outside message black p...,3,0.466667,16,3.281250,3.006793,0.144188,0.757131,0.098675
...,...,...,...,...,...,...,...,...,...,...,...
95,95,Care career,Word month trial.,3,0.133333,19,2.789474,2.817451,0.085879,0.775926,0.138184
96,96,Everyone not forward,Middle prevent discover.,6,0.866667,17,2.411765,2.669426,0.109412,0.782488,0.108135
97,97,Choice need,Themselves like again way.,6,0.466667,20,2.700000,2.777942,0.077640,0.705680,0.216675
98,98,Need sit father,Left clear exist piece because full.,6,0.466667,23,2.543478,2.699700,0.147761,0.679687,0.172539


In [29]:
import nltk
nltk.download('punkt')
prod_info_final['textFeature_token'] = prod_info_final.apply(lambda row: nltk.word_tokenize(row['name']+" "+row['description']), axis=1)
prod_info_final['textFeature'] = prod_info_final.apply(lambda row: row['name']+" "+row['description'], axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [30]:
EMBEDDING_FILE = '/content/GoogleNews-vectors-negative300.bin.gz'
!wget -P /content/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-07-27 14:40:51--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.194.24
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.194.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  47.4MB/s    in 34s     

2021-07-27 14:41:26 (45.7 MB/s) - ‘/content/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [31]:
from gensim import models
w2v_model = models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [32]:
from sklearn.neighbors import NearestNeighbors

from gensim.models import Word2Vec,doc2vec
from collections import namedtuple
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def vectorize(doc,w2v_model=w2v_model,stop_words=stop_words):
  doc = doc.lower()
  words = [w for w in doc.split(" ") if w not in stop_words]
  word_vecs = []
  for word in words:
      try:
          vec = w2v_model[word]
          word_vecs.append(vec)
      except KeyError:
          pass
  vector = np.mean(word_vecs, axis=0)
  return vector


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [33]:
target_docs = prod_info_final['textFeature'].to_list()
X=[]
for i in target_docs:
  X.append(vectorize(i))

In [34]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X)

In [35]:
distances, indices = nbrs.kneighbors(X)

In [36]:
distances_list=distances.tolist()
indices_list=indices.tolist()

In [37]:
import itertools

indices_distances=[]
for i in range(len(distances)):
 indices_distances.append(list(zip(indices_list[i],distances_list[i])))


# Sorting best product for user

Input: user id and product ids

In [38]:
uid=8
pids=[6,43,74]

In [39]:
def recommandation(uid,pids,uhis_info_path):
  recommanded=[]
  for i in pids:
      recommanded.extend(indices_distances[i])
  r_products={}
  for prod in recommanded:
    if prod[0] in r_products:
      r_products[prod[0]] = min(r_products[prod[0]],prod[1])
    else:
      r_products[prod[0]] = prod[1]
  uhis_info=pd.read_csv(uhis_info_path,index_col=0)
  userhistory=uhis_info[uhis_info['userId']==uid].groupby(["prodId"],as_index=False).agg(visit_count=pd.NamedAgg(column="prodId", aggfunc="count"))
  oldvisited=userhistory.set_index('prodId')['visit_count'].to_dict()
  fr_products={}
  print(r_products)
  for key,value in r_products.items():
    if key not in pids:
      if key in oldvisited.keys():
        fr_products[key]=abs(value-max(r_products.values()))*oldvisited[key]
      else:
        fr_products[key]=abs(value-max(r_products.values()))
  return fr_products
fr_products=recommandation(uid,pids,uhis_info_path=uhis_info_path)

{6: 0.0, 78: 1.064142627164712, 12: 1.011751202236738, 48: 0.9358484715960574, 62: 1.1583103620666755, 70: 1.1608897268195257, 46: 1.1680919954294047, 93: 1.1750820623234144, 88: 1.1778507492431387, 74: 0.0, 43: 0.0, 75: 1.129849791354836, 58: 1.178765423555653, 28: 1.2053756009078211, 80: 1.0527061242516784, 59: 1.2518304114107779, 7: 1.2809451433214953, 51: 0.9561516915704732, 68: 1.0395589211410252, 83: 1.0463181843785267, 69: 1.0564105461125117, 41: 1.0757146514878269}


In [40]:
fr_productsdf=pd.DataFrame(list(fr_products.items()),columns=['prodId','similarity'])

In [41]:
fr_productsdf['similarity'] =(fr_productsdf['similarity'] - fr_productsdf['similarity'].min()) / (fr_productsdf['similarity'].max() - fr_productsdf['similarity'].min())


In [42]:
fr_productsdf['similarity'] = fr_productsdf['similarity'].apply(lambda x: x*0.2)

In [43]:
fr_productsdf=fr_productsdf.merge(prod_info_final,on='prodId')[['prodId','similarity','product_visitor_count','rscore','positive']]
fr_productsdf['product_visitor_count'] = fr_productsdf['product_visitor_count'].apply(lambda x: x*0.1)
fr_productsdf['rscore'] =(fr_productsdf['rscore'] - fr_productsdf['rscore'].min()) / (fr_productsdf['rscore'].max() - fr_productsdf['rscore'].min())
fr_productsdf['rscore'] = fr_productsdf['rscore'].apply(lambda x: x*0.3)
fr_productsdf['positive'] = fr_productsdf['positive'].apply(lambda x: x*0.4)
fr_productsdf['Final_score'] = fr_productsdf.iloc[:, -4:-1].sum(axis=1)
fr_productsdf['Final_score'] = fr_productsdf['Final_score'].apply(lambda x: x*100)

In [44]:
recommanded_prodcuts=fr_productsdf.sort_values('Final_score',ascending=False)[['prodId','Final_score']]
recommanded_prodcuts

,prodId,Final_score
0,78,46.945729
7,88,41.974812
18,41,41.724964
1,12,38.286348
2,48,36.685366
4,70,34.254518
14,51,32.271379
16,83,29.141814
9,58,28.949743
17,69,28.737490


In [45]:
prod_dict = []

for index, row in list(recommanded_prodcuts.iterrows()):
    prod_dict.append(dict(row))
prod_dict

[{'Final_score': 46.94572862935972, 'prodId': 78.0},
 {'Final_score': 41.97481242359703, 'prodId': 88.0},
 {'Final_score': 41.724964273132535, 'prodId': 41.0},
 {'Final_score': 38.28634813631401, 'prodId': 12.0},
 {'Final_score': 36.68536617314504, 'prodId': 48.0},
 {'Final_score': 34.25451784712854, 'prodId': 70.0},
 {'Final_score': 32.27137878500139, 'prodId': 51.0},
 {'Final_score': 29.141814464883108, 'prodId': 83.0},
 {'Final_score': 28.949743055060427, 'prodId': 58.0},
 {'Final_score': 28.73748976613259, 'prodId': 69.0},
 {'Final_score': 28.51399810733365, 'prodId': 75.0},
 {'Final_score': 26.04598460948635, 'prodId': 80.0},
 {'Final_score': 24.733701085453056, 'prodId': 68.0},
 {'Final_score': 22.643909009847228, 'prodId': 62.0},
 {'Final_score': 20.899825476053646, 'prodId': 93.0},
 {'Final_score': 14.8701132610581, 'prodId': 46.0},
 {'Final_score': 12.220641156575532, 'prodId': 28.0},
 {'Final_score': 10.278792289673694, 'prodId': 7.0},
 {'Final_score': 5.6873377401843745, 'pr